In [4]:
import json
from collections import Counter
from keras.models import Model
from keras.layers import Embedding,Input,Reshape
from keras.layers.merging import Dot
from sklearn.linear_model import LinearRegression
import numpy as np
import random
from sklearn import svm

In [5]:
with open('data/wp_movies_second.ndjson') as fin:
    movies = [json.loads(l) for l in fin]
# with open('wp_movies_10k.ndjson') as fin:
#     movies = fin.read()
#     print(movies)
#     mv2 = json.loads(movies)

In [6]:
link_counts = Counter()
for movie in movies:
    link_counts.update(movie[2])
link_counts.most_common(10)

[('Rotten Tomatoes', 9393),
 ('Category:English-language films', 5882),
 ('Category:American films', 5867),
 ('Variety (magazine)', 5450),
 ('Metacritic', 5112),
 ('Box Office Mojo', 4186),
 ('The New York Times', 3818),
 ('The Hollywood Reporter', 3553),
 ('Roger Ebert', 2707),
 ('Los Angeles Times', 2454)]

In [8]:
top_links = [link for link,c in link_counts.items() if c>=3]
link_to_idx = {link:idx for idx, link in enumerate(top_links)}
movie_to_idx = {movie[0]:idx for idx,movie in enumerate(movies)}
pairs=[]
for movie in movies:
    pairs.extend((link_to_idx[link],movie_to_idx[movie[0]]) for link in movie[2] if link in link_to_idx)
pairs_set = set(pairs)
len(pairs),len(top_links),len(movie_to_idx)

(949544, 66913, 10000)

In [9]:
def movie_embedding_model(embedding_size=50):
    link = Input(name='link',shape=(1,))
    movie = Input(name='movie', shape=(1,))
    link_embedding = Embedding(name="link_embedding", input_dim=len(top_links),output_dim=embedding_size)(link)
    movie_embedding = Embedding(name="movie_embedding", input_dim=len(movie_to_idx),output_dim=embedding_size)(movie)
    dot = Dot(name='dot_product',normalize=True,axes=2)([link_embedding, movie_embedding])
    merged = Reshape((1,))(dot)
    model = Model(inputs=[link,movie],outputs=[merged])
    model.compile(optimizer='nadam', loss='mse')
    return model

In [10]:
model = movie_embedding_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 link (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 movie (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 link_embedding (Embedding)     (None, 1, 50)        3345650     ['link[0][0]']                   
                                                                                                  
 movie_embedding (Embedding)    (None, 1, 50)        500000      ['movie[0][0]']                  
                                                                                              

In [12]:
random.seed(5)

def batchifier(pairs,positive_samples=50,negative_ratio=10):
    batch_size = positive_samples*(1+negative_ratio)
    batch = np.zeros((batch_size,3))
    while True:
        for idx,(link_id,movie_id) in enumerate(random.sample(pairs,positive_samples)):
            batch[idx,:]=(link_id,movie_id,1)
        idx = positive_samples
        while idx<batch_size:
            movie_id=random.randrange(len(movie_to_idx))
            link_id=random.randrange(len(top_links))
            if not (link_id,movie_id) in pairs_set:
                batch[idx,:]=(link_id,movie_id,-1)
                idx += 1
        np.random.shuffle(batch)
        yield {'link':batch[:,0],'movie':batch[:,1]},batch[:,2]

next(batchifier(pairs,positive_samples=3,negative_ratio=2))

({'link': array([13365., 20558., 31254., 32318., 48731.,  3801.,  1313., 32643.,
         22418.]),
  'movie': array([6238.,  849., 5530., 7685., 1854., 5874., 7236., 7628., 1529.])},
 array([-1., -1.,  1., -1., -1., -1.,  1., -1.,  1.]))

In [ ]:
positive_samples_per_batch = 512

model.fit_generator(
    batchifier(pairs,positive_samples=positive_samples_per_batch,negative_ratio=10),
    epochs=15,
    steps_per_epoch=len(pairs),
    verbose=2
)

Epoch 1/15


<ipython-input-13-27e8724503c9>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


In [ ]:
movie = model.get_layer('movie_embedding')
movie_weights = movie.get_weights()[0]
movie_lengths = np.linalg.norm(movie_weights,axis=1)
normalized_movies = (movie_weights.T/movie_lengths).T

def similar_movies(movie):
    dist = np.dot(normalized_movies,normalized_movies[movie_to_idx[movie]])
    closest = np.argsort(dist)[-10:]
    for c in reversed(closest):
        print(c,movies[c][0],dist[c])

similar_movies('Rouge One')

In [ ]:
link = model.get_layer('link_embedding')
link_weights = link.get_weights()[0]
link_lengths = np.linalg.norm(link_weights,axis=1)
normalized_links = (link_weights.T/link_lengths).T

def similar_links(link):
    dist = np.dot(normalized_links,normalized_links[link_to_idx[link]])
    closest = np.argsort(dist)[-10:]
    for c in reversed(closest):
        print(c,top_links[c][0],dist[c])

similar_links('George Lucas')